In [1]:
import requests
import pandas as pd
import xml.etree.ElementTree as ET
import warnings
import math
from tqdm import tqdm
from irsx.xmlrunner import XMLRunner
xml_runner = XMLRunner()
warnings.filterwarnings('ignore')

import ipyparallel as ipp
c = ipp.Client()
print(c.ids)
dview = c[:] #So this dview (direct viewing) becomes the parallelizing program-direct execution in cloud

[0, 1, 2, 3]


In [2]:
import re

In [3]:
einlist = pd.read_csv("/Users/Wolfalev/Documents/Grad/UT-AUSTIN GRAD/Coursework/CUSS/JiRepo/npo_classifier/dataset/union_ein_name_list.csv")

#for ein in einlist['EIN']:
 #   print(ein)

In [4]:
#xml_runner = XMLRunner()
#parsed_filing = xml_runner.run_sked(2012134249347, 'IRS990ScheduleJ')

### Need to get object ID

### Prepare UK Data.

```Python
# Compress icnpo training data.
df_icnpo_classifier_training_data=pd.read_csv('../dataset/icnpo_classifier_training_data.csv', sep=',')
df_icnpo_classifier_training_data.to_pickle('../dataset/icnpo_classifier_training_data.pkl.gzip', compression='gzip')
```

### Prepare US Training Data.
Useful links:
- IRS 990 forms on AWS: https://registry.opendata.aws/irs990/
- NCCS Data Archive (data dict, NTEE codes, etc.): https://nccs-data.urban.org/index.php

### TODO:
- [+] Add year of formation. - Do not use information from 990 forms: only Form 990 has "Year of Formation". Use BMF 199508 to dummy-code the organizations.
- [ ] Add program descriptions in Part VIII of Form 990.

In [5]:
df_index=pd.DataFrame()
for year in range(2013, 2019):
        try:
            print(str(year), 'started', end='\t')
            df_index_temp=pd.read_csv('https://s3.amazonaws.com/irs-form-990/index_'+str(year)+'.csv', 
                                  error_bad_lines=False, warn_bad_lines=True # Be cautious with bad lines.
                                 )
            df_index_temp['YEAR']=year
            df_index=pd.concat([df_index, df_index_temp], ignore_index=True)
            print('got')
        except:
            print('passed')

2013 started	got
2014 started	

b'Skipping line 39569: expected 9 fields, saw 10\n'


got
2015 started	got
2016 started	got
2017 started	got
2018 started	got


In [6]:
df_index.groupby(['YEAR','RETURN_TYPE']).count()

RETURN_ID  FILING_TYPE     EIN  TAX_PERIOD  SUB_DATE  \
YEAR RETURN_TYPE                                                         
2013 990             115531       115531  115531      115531    115531   
     990EO            21991        21991   21991       21991     21991   
     990EZ            60484        60484   60484       60484     60484   
     990O             38029        38029   38029       38029     38029   
     990PF            25414        25414   25414       25414     25414   
2014 990             163381       163381  163381      163381    163381   
     990EO            29466        29466   29466       29466     29466   
     990EZ            82937        82937   82937       82937     82937   
     990O             52490        52490   52490       52490     52490   
     990PF            59254        59254   59254       59254     59254   
2015 990             106571       106571  106571      106571    106571   
     990EO            21399        21399   21399       21399     21399   
     990EZ            59845        59845   59845       59845     59845   
     990O             34567        34567   34567       34567     34567   
     990PF            38650        38650   38650       38650     38650   
2016 990             160550       160550  160550      160550    160550   
     990EO            28537        28537   28537       28537     28537   
     990EZ            84652        84652   84652       84652     84652   
     990O             50987        50987   50987       50987     50987   
     990PF            53694        53694   53694       53694     53694   
2017 990             207734       207734  207734      207734    207734   
     990EO            36318        36318   36318       36318     36318   
     990EZ           109648       109648  109648      109648    109648   
     990O             66766        66766   66766       66766     66766   
     990PF            68547        68547   68547       68547     68547   
2018 990             192489       192489  192489      192489    192489   
     990EO            32690        32690   32690       32690     32690   
     990EZ           103694       103694  103694      103694    103694   
     990O             61005        61005   61005       61005     61005   
     990PF            67632        67632   67632       67632     67632   

                  TAXPAYER_NAME     DLN  OBJECT_ID  
YEAR RETURN_TYPE                                    
2013 990                 115531  115531     115531  
     990EO                21991   21991      21991  
     990EZ                60484   60484      60484  
     990O                 38029   38029      38029  
     990PF                25414   25414      25414  
2014 990                 163381  163381     163381  
     990EO                29466   29466      29466  
     990EZ                82937   82937      82937  
     990O                 52490   52490      52490  
     990PF                59254   59254      59254  
2015 990                 106571  106571     106571  
     990EO                21399   21399      21399  
     990EZ                59845   59845      59845  
     990O                 34567   34567      34567  
     990PF                38650   38650      38650  
2016 990                 160550  160550     160550  
     990EO                28537   28537      28537  
     990EZ                84652   84652      84652  
     990O                 50987   50987      50987  
     990PF                53694   53694      53694  
2017 990                 207734  207734     207734  
     990EO                36318   36318      36318  
     990EZ               109648  109648     109648  
     990O                 66766   66766      66766  
     990PF                68547   68547      68547  
2018 990                 192489  192489     192489  
     990EO                32690   32690      32690  
     990EZ               103694  103694     103694  
     990O                 61005   61005      61005  
     9

### Acquiring text data.

In [7]:
df_index_matched = df_index[df_index['EIN'].isin(einlist['EIN'])]
                       
len(df_index_matched)                                               

3024

In [8]:
df_index_matched.sample(10)

,RETURN_ID,FILING_TYPE,EIN,TAX_PERIOD,SUB_DATE,TAXPAYER_NAME,RETURN_TYPE,DLN,OBJECT_ID,YEAR
579666,11764215,EFILE,30453788,201312,9/15/2014 10:44:20 PM,FRIENDS OF DEEP EDDY INC,990EZ,93492122008574,201421229349200857,2014
280681,11108053,EFILE,870699956,201212,1/3/2014,TOWN LAKE TRAIL FOUNDATION,990,93493319039623,201323199349303962,2014
16490,11072404,EFILE,270232323,201212,12/26/2013,TURQUOISE COUNCIL OF AMERICANS AND EURASIANS,990,93493318025383,201333189349302538,2013
1519356,14686592,EFILE,742757165,201612,8/29/2017 11:27:54 PM,TEXAS COALITION GLCI INC,990,93493118009557,201701189349300955,2017
937396,13131934,EFILE,751886520,201412,01/21/2016,TEXAS TREES FOUNDATION,990,93493317082115,201513179349308211,2016
1887745,15245384,EFILE,747055759,201708,4/4/2018 5:01:40 PM,KEEP TEXAS BEAUTIFUL INC,990,93493067006298,201840679349300629,2018
870774,12742890,EFILE,205796835,201412,8/3/2015,JACQUELINE DAVIS MINISTRIES,990EZ,93492076003155,201500769349200315,2015
1002424,13136192,EFILE,741708558,201412,01/22/2016,GREENWOOD FOREST FUND INC,990O,93493317045025,201523179349304502,2016
1836071,15156484,EFILE,721621543,201706,1/31/2018 6:54:22 AM,CFA SOCIETY OF AUSTIN,990EO,93492313015197,201743139349201519,2018
2010662,15656767,EFILE,464275961,201712,9/6/2018 7:44:15 AM,ACCION MARTIN HOLDINGS INC,990O,93493145005228,201821459349300522,2018


In [9]:
done_index_list=[]
dview.execute('import pandas as pd')
dview.execute('from time import sleep')
dview.execute('from irsx.xmlrunner import XMLRunner')
dview.execute('import os')
dview['xml_runner'] = XMLRunner()
dview['df_index'] = df_index_matched
dview['df_index_text_acq'] = pd.DataFrame()

In [10]:
dview['done_index_list'] = list(set(done_index_list))
dview['counter']=0

In [11]:
###### Define functions ######################################
def func_irs990_text(obj_id):
    IRS990_result=xml_runner.run_sked(obj_id, 'IRS990').result 

    ############## Check filing version.##############
    if IRS990_result!=None:
        IRS990_result=IRS990_result[0]
    else:
        return {'IRS990_p1_ActvtyOrMssnDsc': 'VERSION_NOT_SUPPORTED',
                'IRS990_p3_MssnDsc': 'VERSION_NOT_SUPPORTED',
                'IRS990_p3_DscS': 'VERSION_NOT_SUPPORTED',
                'IRS990ScheduleO_ExplntnTxt': 'VERSION_NOT_SUPPORTED',
               }
    ##################################################

    ###### IRS990_p1_ActvtyOrMssnDsc #################
    IRS990_p1_ActvtyOrMssnDsc=IRS990_result['schedule_parts']['part_i']['ActvtyOrMssnDsc']
    ##################################################

    ##### IRS990_p3_MssnDsc, IRS990_p3_DscS ##########
    IRS990_p3=IRS990_result['schedule_parts']['part_iii']
    IRS990_p3_MssnDsc='##'.join([IRS990_p3[key] for key in IRS990_p3.keys() if key=='MssnDsc'])
    IRS990_p3_DscS='##'.join([IRS990_p3[key] for key in IRS990_p3.keys() if (key=='Dsc' or '_Dsc' in key)])
    ##################################################

    ###### IRS990ScheduleO_ExplntnTxt #################
#         if 'InfInSkdOPrtIIIInd' in IRS990_p3.keys() and IRS990_p3['InfInSkdOPrtIIIInd']=='X' and 'IRS990ScheduleO' in xml_runner.run_filing(obj_id).list_schedules():
    if 'IRS990ScheduleO' in xml_runner.run_filing(obj_id).list_schedules():
        IRS990ScheduleO=xml_runner.run_sked(obj_id, 'IRS990ScheduleO').result[0]
        IRS990ScheduleO_ExplntnTxt='##'.join([s['ExplntnTxt'] for s in IRS990ScheduleO['groups']['SkdOSpplmntlInfrmtnDtl'] if 'III' in s['FrmAndLnRfrncDsc']])
    else:
        IRS990ScheduleO_ExplntnTxt=''
    ##################################################

    return {'IRS990_p1_ActvtyOrMssnDsc': IRS990_p1_ActvtyOrMssnDsc,
            'IRS990_p3_MssnDsc': IRS990_p3_MssnDsc,
            'IRS990_p3_DscS': IRS990_p3_DscS,
            'IRS990ScheduleO_ExplntnTxt': IRS990ScheduleO_ExplntnTxt,
           }

def func_irs990ez_text(obj_id):
    IRS990EZ_result=xml_runner.run_sked(obj_id, 'IRS990EZ').result

    ############## Check filing version.##############
    if IRS990EZ_result!=None:
        IRS990EZ_result=IRS990EZ_result[0]
    else:
        return {'IRS990EZ_p3_PrmryExmptPrpsTxt': 'VERSION_NOT_SUPPORTED',
                'IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt': 'VERSION_NOT_SUPPORTED',
                'IRS990ScheduleO_ExplntnTxt': 'VERSION_NOT_SUPPORTED',
               }
    ########################################################

    ###### IRS990EZ_p3_PrmryExmptPrpsTxt #############
    if 'ez_part_iii' in IRS990EZ_result['schedule_parts'].keys():
        IRS990EZ_p3=IRS990EZ_result['schedule_parts']['ez_part_iii']
        IRS990EZ_p3_PrmryExmptPrpsTxt='##'.join([IRS990EZ_p3[key] for key in IRS990EZ_p3.keys() if key=='PrmryExmptPrpsTxt'])
    else:
        IRS990EZ_p3_PrmryExmptPrpsTxt=''
    ##################################################

    ####### IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt ######
    if 'EZPrgrmSrvcAccmplshmnt' in IRS990EZ_result['groups'].keys():
        IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt='##'.join([s['DscrptnPrgrmSrvcAccmTxt'] for s in IRS990EZ_result['groups']['EZPrgrmSrvcAccmplshmnt']])
    else:
        IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt=''
    ##################################################

    ###### IRS990ScheduleO_ExplntnTxt #################
#         if 'InfInSkdOPrtIIIInd' in IRS990EZ_p3.keys() and IRS990EZ_p3['InfInSkdOPrtIIIInd']=='X' and 'IRS990ScheduleO' in xml_runner.run_filing(obj_id).list_schedules():
    if 'IRS990ScheduleO' in xml_runner.run_filing(obj_id).list_schedules():
        IRS990ScheduleO=xml_runner.run_sked(obj_id, 'IRS990ScheduleO').result[0]
        IRS990ScheduleO_ExplntnTxt='##'.join([s['ExplntnTxt'] for s in IRS990ScheduleO['groups']['SkdOSpplmntlInfrmtnDtl'] if 'III' in s['FrmAndLnRfrncDsc']])
    else:
        IRS990ScheduleO_ExplntnTxt=''
    ##################################################

    return {'IRS990EZ_p3_PrmryExmptPrpsTxt': IRS990EZ_p3_PrmryExmptPrpsTxt,
            'IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt': IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt,
            'IRS990ScheduleO_ExplntnTxt': IRS990ScheduleO_ExplntnTxt,
           }

def func_irs990pf_text(obj_id):
    IRS990PF_result=xml_runner.run_sked(obj_id, 'IRS990PF').result

    ############## Check filing version.##############
    if IRS990PF_result!=None:
        IRS990PF_result=IRS990PF_result[0]
    else:
        return {'IRS990PF_p9a_DscrptnTxt': 'VERSION_NOT_SUPPORTED',
                'IRS990PF_p16b_RltnshpSttmntTxt': 'VERSION_NOT_SUPPORTED',
               }
    ########################################################

    ##### IRS990PF_p9a_DscrptnTxt ##########################
    if 'pf_part_ixa' in IRS990PF_result['schedule_parts'].keys():
        IRS990PF_p9a=IRS990PF_result['schedule_parts']['pf_part_ixa']
        IRS990PF_p9a_DscrptnTxt='##'.join([IRS990PF_p9a[key] for key in IRS990PF_p9a.keys() if 'Txt' in key])
    else:
        IRS990PF_p9a_DscrptnTxt=''
    ########################################################

    ##### IRS990PF_p16b_RltnshpSttmntTxt ###################
    if 'PFRlnOfActyTAccmOfExmptPrps' in IRS990PF_result['groups']:
        IRS990PF_p16b_RltnshpSttmntTxt='##'.join([s['RltnshpSttmntTxt'] for s in IRS990PF_result['groups']['PFRlnOfActyTAccmOfExmptPrps']])
    else:
        IRS990PF_p16b_RltnshpSttmntTxt=''
    ########################################################

    return {'IRS990PF_p9a_DscrptnTxt':IRS990PF_p9a_DscrptnTxt,
            'IRS990PF_p16b_RltnshpSttmntTxt':IRS990PF_p16b_RltnshpSttmntTxt,
           }
###### Define functions ######################################

In [12]:
###### Run main function ################################
df_index_text_acq = pd.DataFrame()
df_index = pd.DataFrame()
df_index = df_index_matched
done_index_list = []
for index in df_index_matched.index:
    try:
        if index not in done_index_list: #Index is position of filing in list of all filings in index file
            obj_id = df_index.loc[index, 'OBJECT_ID'] #if that position is not filled, pull the object id
            return_type=df_index.loc[index, 'RETURN_TYPE'] #if that position is not filled, pull the return type
            if return_type in ['990', '990O']: # (1) if return type  is 990, do the following
                text_dict=func_irs990_text(obj_id) #apply this function for 990s, using the object id from index to get the 990 in a text_dict
                df_index.loc[index, 'IRS990_p1_ActvtyOrMssnDsc']=text_dict['IRS990_p1_ActvtyOrMssnDsc'] #get mission and put it in index file
                df_index.loc[index, 'IRS990_p3_MssnDsc']=text_dict['IRS990_p3_MssnDsc'] # get mssndsc and put it here
                df_index.loc[index, 'IRS990_p3_DscS']=text_dict['IRS990_p3_DscS'] # get DsC and put it here
                df_index.loc[index, 'IRS990ScheduleO_ExplntnTxt']=text_dict['IRS990ScheduleO_ExplntnTxt']# get schedule 0 and put it here
            elif return_type in ['990EZ', '990EO']: #(2) same thing
                text_dict=func_irs990ez_text(obj_id)
                df_index.loc[index, 'IRS990EZ_p3_PrmryExmptPrpsTxt']=text_dict['IRS990EZ_p3_PrmryExmptPrpsTxt']
                df_index.loc[index, 'IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt']=text_dict['IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt']
                df_index.loc[index, 'IRS990ScheduleO_ExplntnTxt']=text_dict['IRS990ScheduleO_ExplntnTxt']
            elif return_type in ['990PF']: #(3) same thing
                text_dict=func_irs990pf_text(obj_id)
                df_index.loc[index, 'IRS990PF_p9a_DscrptnTxt']=text_dict['IRS990PF_p9a_DscrptnTxt']
                df_index.loc[index, 'IRS990PF_p16b_RltnshpSttmntTxt']=text_dict['IRS990PF_p16b_RltnshpSttmntTxt']
            df_index_text_acq=pd.concat([df_index_text_acq, df_index.loc[[index]]], ignore_index=True) #take the entries we just pulled, turn into df for that row, and concatenate them together into df_index_text_acq
            done_index_list+=[index] ## update that we've done this many indicies. 
    except:
        pass

Filing version 2012v2.0 isn't supported for this operation
Filing version 2012v2.1 isn't supported for this operation
Filing version 2012v2.1 isn't supported for this operation
Filing version 2012v2.1 isn't supported for this operation
Filing version 2012v2.1 isn't supported for this operation
Filing version 2012v2.3 isn't supported for this operation
Filing version 2012v2.1 isn't supported for this operation
Filing version 2012v2.1 isn't supported for this operation
Filing version 2011v1.5 isn't supported for this operation
Filing version 2012v2.1 isn't supported for this operation
Filing version 2011v1.5 isn't supported for this operation
Filing version 2012v2.1 isn't supported for this operation
Filing version 2012v2.0 isn't supported for this operation
Filing version 2012v2.0 isn't supported for this operation
Filing version 2011v1.3 isn't supported for this operation
Filing version 2012v2.1 isn't supported for this operation
Filing version 2012v2.2 isn't supported for this operati

In [13]:
len(set(done_index_list)), len(df_index_text_acq)

(3020, 3020)

In [14]:
df_index_text_acq.sample(10)

,DLN,EIN,FILING_TYPE,IRS990EZ_p3_DscrptnPrgrmSrvcAccmTxt,IRS990EZ_p3_PrmryExmptPrpsTxt,IRS990PF_p16b_RltnshpSttmntTxt,IRS990PF_p9a_DscrptnTxt,IRS990ScheduleO_ExplntnTxt,IRS990_p1_ActvtyOrMssnDsc,IRS990_p3_DscS,IRS990_p3_MssnDsc,OBJECT_ID,RETURN_ID,RETURN_TYPE,SUB_DATE,TAXPAYER_NAME,TAX_PERIOD,YEAR
333,93493226032263,742934364,EFILE,NaN,NaN,NaN,NaN,VERSION_NOT_SUPPORTED,VERSION_NOT_SUPPORTED,VERSION_NOT_SUPPORTED,VERSION_NOT_SUPPORTED,201312269349303226,11002035,990,12/6/2013,SOLAR SAN ANTONIO,201212,2013
646,93491302001014,760447921,EFILE,NaN,NaN,,,NaN,NaN,NaN,NaN,201413029349100101,12012380,990PF,11/25/2014 8:32:42 PM,GOOD EARTH FOUNDATION,201404,2014
187,93493182009573,142003280,EFILE,NaN,NaN,NaN,NaN,VERSION_NOT_SUPPORTED,VERSION_NOT_SUPPORTED,VERSION_NOT_SUPPORTED,VERSION_NOT_SUPPORTED,201321829349300957,10850223,990,9/20/2013,ENVIRONMENTAL STEWARDSHIP,201212,2013
2250,93493129018527,460920592,EFILE,NaN,NaN,NaN,NaN,,PROVIDE SHELTER AND CARE AND SPAY AND NEUTER S...,SPAY AND NEUTER ANIMALS,SPAY AND NEUTER ANIMALS AND PROVIDE SHELTER AN...,201721299349301852,14826060,990,10/12/2017 8:33:56 AM,TEXAS LITTER CONTROL,201612,2017
116,93493141005153,651201715,EFILE,NaN,NaN,NaN,NaN,VERSION_NOT_SUPPORTED,VERSION_NOT_SUPPORTED,VERSION_NOT_SUPPORTED,VERSION_NOT_SUPPORTED,201301419349300515,10722693,990,7/31/2013,BURKBURNETT GRACE MINISTRIES INC,201212,2013
1296,93493317068675,251907558,EFILE,NaN,NaN,NaN,NaN,THE GEAA TECHNICAL RESEARCH PROGRAM PROVIDES O...,THE GREATER EDWARDS AQUIFER ALLIANCE (GEAA) IS...,CAPACITY BUILDING - GEAA STAFF PROVIDES MEMBER...,THE GREATER EDWARDS AQUIFER ALLIANCE (GEAA) IS...,201523179349306867,13142695,990,01/26/2016,GREATER EDWARDS AQUIFER ALLIANCE,201412,2016
2321,93493262005057,750771007,EFILE,NaN,NaN,NaN,NaN,OTHER PROGRAM SERVICES 4: MEMBERSHIP PLAQUES O...,TEXAS FORESTRY ASSOCIATION'S MISSION IS TO PRO...,MEETING EXPENSES - ADVANCING THE INTEREST IN A...,TEXAS FORESTRY ASSOCIATION'S MISSION IS TO PRO...,201702629349300505,14927589,990O,11/6/2017 7:57:21 PM,TEXAS FORESTRY ASSOCIATION INC,201612,2017
2798,93492213008138,472840205,EFILE,FINANCIAL ASSISTANCE TO VARIOUS LOCAL EL PASO ...,TO ASSIT THE EL PASO COMMUNITY BY HELPING RELI...,NaN,NaN,,NaN,NaN,NaN,201832139349200813,15733315,990EZ,9/26/2018 2:13:14 PM,EL PASO CHIHUAHUAS FOUNDATION,201712,2018
2778,93493219003278,273886247,EFILE,NaN,NaN,NaN,NaN,,PROVIDE SOLAR BASED LED LIGHTING SYSTEM AS ONE...,Solar Lighting for poverished areas,PROVIDE SOLAR BASED LED LIGHTING SYSTEM AS ONE...,201822199349300327,15779163,990,10/10/2018 11:25:32 PM,ECOSOLUTIONS FOUNDATION INC,201712,2018
2108,93491090016757,760265720,EFILE,NaN,NaN,THIS ORGANIZATION IS LISTED AS A 501(C)(3) ORG...,"KLEINWOOD CHURCH OF CHRIST##JUSTIN DOBBS, MINI...",NaN,NaN,NaN,NaN,201700909349101675,14659394,990PF,8/19/2017 12:39:39 AM,JL HODGES FOUNDATION,201612,2017


In [15]:
#df_index_text_acq_2018= df_index_text_acq[df_index_text_acq.YEAR == 2018]
#All Years

In [16]:
df_index_text_acq.sample(10)
print(len(df_index_text_acq))

3020


len(df_EIN_TXT), len(df_index_matched)

In [ ]:
df_bmf_1812=pd.read_csv('https://nccs-data.urban.org/data/bmf/2018/bmf.bm1812.csv')
df_bmf_1712=pd.read_csv('https://nccs-data.urban.org/data/bmf/2017/bmf.bm1712.csv')
df_bmf_1608=pd.read_csv('https://nccs-data.urban.org/data/bmf/2016/bmf.bm1608.csv')
df_bmf_1512=pd.read_csv('https://nccs-data.urban.org/data/bmf/2015/bmf.bm1512.csv')
df_bmf_1412=pd.read_csv('https://nccs-data.urban.org/data/bmf/2014/bmf.bm1412.csv')
df_bmf_1312=pd.read_csv('https://nccs-data.urban.org/data/bmf/2013/bmf.bm1312.csv')

In [ ]:
df_bmf_1812['YEAR_NCCS'] = 2018
df_bmf_1712['YEAR_NCCS'] = 2017
df_bmf_1608['YEAR_NCCS'] = 2016
df_bmf_1512['YEAR_NCCS'] = 2015
df_bmf_1412['YEAR_NCCS'] = 2014
df_bmf_1312['YEAR_NCCS'] = 2013

#df_bmf_1812['Real_Year'] = df_bmf_1812['TAXPER'].astype(str).str[:4]
#df_bmf_1712['Real_Year'] = df_bmf_1712['TAXPER'].astype(str).str[:4]
#df_bmf_1608['Real_Year'] = df_bmf_1608['TAXPER'].astype(str).str[:4]
#df_bmf_1512['Real_Year'] = df_bmf_1512['TAXPER'].astype(str).str[:4]
#df_bmf_1412['Real_Year'] = df_bmf_1412['TAXPER'].astype(str).str[:4]
#df_bmf_1312['Real_Year'] = df_bmf_1312['TAXPER'].astype(str).str[:4]

#df_bmf_1812 = df_bmf_1812[df_bmf_1812['Real_Year'] == '2018']
#df_bmf_1712 = df_bmf_1712[df_bmf_1712['Real_Year'] == '2017']
#df_bmf_1608 = df_bmf_1608[df_bmf_1608['Real_Year'] == '2016']
#df_bmf_1512 = df_bmf_1512[df_bmf_1512['Real_Year'] == '2015']
#df_bmf_1412 = df_bmf_1412[df_bmf_1412['Real_Year'] == '2014']
#df_bmf_1312 = df_bmf_1312[df_bmf_1312['Real_Year'] == '2013']



In [ ]:
print(len(df_bmf_1812))
print(len(df_bmf_1712))
print(len(df_bmf_1608))
print(len(df_bmf_1512))
print(len(df_bmf_1412))
print(len(df_bmf_1312))

In [ ]:
#df_bmf_1812.drop_duplicates(subset=['TAXPER'],keep='last')
#df_bmf_1712.drop_duplicates(subset='EIN',keep='last')
#df_bmf_1608.drop_duplicates(subset='EIN',keep='last')
#df_bmf_1512.drop_duplicates(subset='EIN',keep='last')
#df_bmf_1412.drop_duplicates(subset='EIN',keep='last')
#df_bmf_1312.drop_duplicates(subset='EIN',keep='last')

In [ ]:
df_bmf_1812.columns

In [ ]:
print(len(df_bmf_1812))
print(len(df_bmf_1712))
print(len(df_bmf_1608))
print(len(df_bmf_1512))
print(len(df_bmf_1412))
print(len(df_bmf_1312))

In [ ]:
df_bmf_13_18=pd.concat([df_bmf_1312, df_bmf_1412, df_bmf_1512, df_bmf_1608, df_bmf_1712, df_bmf_1812], ignore_index=True)
df_bmf_13_18['Real_Year'] = df_bmf_13_18['TAXPER'].astype(str).str[:4]
df_bmf_13_18= df_bmf_13_18.drop_duplicates(subset=['EIN', 'Real_Year'],keep='last')
#remove any duplicated taxperiods within an ein, only taking  the last


In [ ]:
df_bmf_13_18 = df_bmf_13_18[df_bmf_13_18.STATE == 'TX']
df_bmf_13_18 = df_bmf_13_18[df_bmf_13_18.NTEE1 == 'C']

In [ ]:
df_bmf_13_18['EIN'] = pd.to_numeric(df_bmf_13_18['EIN'])
df_index_text_acq['EIN'] = pd.to_numeric(df_index_text_acq['EIN'])

In [ ]:
#df_bmf_13_18.drop_duplicates(subset=['EIN', 'CTAXPER'],keep='first')
#print(len(df_bmf_13_18))
df_index_text_acq['Real_Year'] = df_index_text_acq['TAX_PERIOD'].astype(str).str[:4]
df_index_text_acq = df_index_text_acq.drop_duplicates(subset=['EIN', 'Real_Year'],keep='last')

print(len(df_index_text_acq))

In [ ]:
#df_bmf_13_18['YEAR_real'] = df_bmf_13_18['CTAXPER']
#df_index_text_acq['YEAR_real'] = df_index_text_acq['TAX_PERIOD']

In [ ]:
df_EIN_TXT=df_index_text_acq.merge(df_bmf_13_18, on=['EIN', 'Real_Year'] , how='outer')
df_EIN_TXT = df_EIN_TXT[df_EIN_TXT.Real_Year != 'nan']
#df_EIN_TXT['unique_id'] = df_EIN_TXT['EIN'] + "_" + df_EIN_TXT['Real_Year']
#df_EIN_TXT.drop_duplicates(subset='unique_id', keep='last')
df_EIN_TXT['Real_Year'] = pd.to_numeric(df_EIN_TXT['Real_Year'])
df_EIN_TXT = df_EIN_TXT[df_EIN_TXT.Real_Year > 2012]

In [ ]:
df_EIN_TXT.sample(3)

In [ ]:
df_EIN_TXT.columns

In [ ]:
print(len(df_EIN_TXT))

In [ ]:
df_EIN_TXT['EIN'].unique

In [ ]:
df_EIN_TXT['EIN'].isin(einlist['EIN']).value_counts()

In [ ]:
#df_EIN_TXT[['EIN', 'TAX_PERIOD']].drop_duplicates()
#len(df_EIN_TXT)

In [ ]:
df_EIN_TXT.to_csv('./texas_20132018_envorgs.csv', index=False)

In [ ]:
df_EIN_TXT_list = df_EIN_TXT[df_EIN_TXT['EIN'].isin(einlist['EIN'])]

In [ ]:
len(df_EIN_TXT_list)

In [ ]:
df_EIN_TXT_list.to_csv('./texas_20132018_envorgs_matched.csv', index=False)